#### Illustration of different hypothetical Pb-loss distributions and their convolution with a Gaussian distribution
##### Accompanyment to "Modeling apparent Pb loss in zircon U-Pb geochronology", submitted to Geochronology (gchron-2023-6)
##### Revision 1 (R1) August 2023
By: Glenn R. Sharman, Department of Geosciences, University of Arkansas

In [ ]:
# Import required modules
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import norm
from scipy.stats import rv_discrete
from scipy.signal import convolve
from scipy.signal import convolve

import convFuncs as convFunc

import matplotlib
%matplotlib inline
%config InlineBackend.figure_format = 'retina' # For improving matplotlib figure resolution
matplotlib.rcParams['pdf.fonttype'] = 42 # For allowing preservation of fonts upon importing into Adobe Illustrator
matplotlib.rcParams['ps.fonttype'] = 42

In [ ]:
# Mean and s.d. of normal distribution
mu = 0 # Mean of normal distribution, in %
omega = 1.5 # Standard deviation of the normal distribution, in %

xdif = 0.01 # X-axis discretization interval, in %
x1 = -100 # Lower x-axis for plotting, in %
x2 = 100 # Upper x-axis for plotting, in %
x = np.arange(x1, x2+xdif, xdif)

In [ ]:
# Define the Gaussian distribution pdf
rv_norm = norm(loc = mu, scale = omega)
norm_pdf = rv_norm.pdf(x)
norm_pdf = norm_pdf/np.sum(norm_pdf) # Normalize so area under the curve = 1

In [ ]:
# Define the parameters for the Pb loss distributions
xk_none = [0]
pk_none = [1]

xk_const = [-3]
pk_const = [1]

xk_mix = [0,-6] # X-axis values in discrete distribution (i.e., Pb loss, as pct)
pk_mix = [0.80, 0.20] # Y-axis values in discrete distribution (i.e., probability of Pb-loss, sum-to-one)

# Define the parameters for the logit-normal distribution
logitnorm_mu = [-3, -5, -5]
logitnorm_omega = [0.1, 1, 2]

# Define the discrete distributions
pb_loss_func_none = rv_discrete(values=(xk_none,pk_none))
pb_loss_func_const = rv_discrete(values=(xk_const,pk_const))
pb_loss_func_mix = rv_discrete(values=(xk_mix,pk_mix))
                                
# Get the cumulative probabilities for the values in x
# This is needed b/c the .pmf() function requires integers
cumulative_probs_none = pb_loss_func_none.cdf(x)
cumulative_probs_const = pb_loss_func_const.cdf(x)
cumulative_probs_mix = pb_loss_func_mix.cdf(x)
                                
# Compute the derivative of the cumulative probabilities so we can get back to pdf space
pdf_none = np.gradient(cumulative_probs_none, xdif)
pdf_const = np.gradient(cumulative_probs_const, xdif)
pdf_mix = np.gradient(cumulative_probs_mix, xdif)
                                
# Normalize so probabilities sum-to-one
pdf_none = pdf_none/np.sum(pdf_none)
pdf_const = pdf_const/np.sum(pdf_const)
pdf_mix = pdf_mix/np.sum(pdf_mix)

# Do the convolution
conv_none = convolve(pdf_none, norm_pdf, mode='same')
conv_const = convolve(pdf_const, norm_pdf, mode='same')
conv_mix = convolve(pdf_mix, norm_pdf, mode='same')

In [ ]:
# Make the figure
x_1 = -15
x_2 = 10

colors = ['red','olive','navy','black']

fig, axs = plt.subplots(4, 4, figsize=(13.3, 10))

axs[0,0].plot(xk_none, pb_loss_func_none.pmf(xk_none), 'ko', ms=12, mec='k')
axs[0,0].vlines(xk_none, 0, pb_loss_func_none.pmf(xk_none), colors='k', lw=4)
axs[0,0].axhline(1.0, ls='--', color='black')
axs[0,0].set_ylim(0,)
axs[0,0].set_xlim(-6.5, 0.5)
axs[0,0].set_ylabel('Relative probability')
axs[0,0].set_title('No Pb loss')

axs[1,0].plot(x, cumulative_probs_none, '-', color='k')
axs[1,0].axhline(1.0, ls='--', color='black')
axs[1,0].set_ylim(0,)
axs[1,0].set_xlim(-6.5, 0.5)
axs[1,0].set_ylabel('Cumulative probability')

axs[2,0].plot(x, norm_pdf, '-', color='black', label='Original')
axs[2,0].plot(x, conv_none, '--', color='red', label='Pb loss')
axs[2,0].set_xlim(x_1, x_2)
axs[2,0].set_ylim(0,)
axs[2,0].set_ylabel('Relative probability')
axs[2,0].legend()

axs[3,0].plot(x, np.cumsum(norm_pdf), '-', color='black', label='Original')
axs[3,0].plot(x, np.cumsum(conv_none), '--', color='red', label='Pb loss')
axs[3,0].set_xlim(x_1, x_2)
axs[3,0].set_ylim(0,1)
axs[3,0].set_xlabel('Pb*/U offset (%)')
axs[3,0].set_ylabel('Cumulative probability')
axs[3,0].legend()

axs[0,1].plot(xk_const, pb_loss_func_const.pmf(xk_const), 'ko', ms=12, mec='k')
axs[0,1].vlines(xk_const, 0, pb_loss_func_const.pmf(xk_const), colors='k', lw=4)
axs[0,1].axhline(1.0, ls='--', color='black')
axs[0,1].set_ylim(0,)
axs[0,1].set_xlim(-6.5, 0.5)
axs[0,1].set_title('Constant Pb loss')

axs[1,1].plot(x, cumulative_probs_const, '-', color='k')
axs[1,1].axhline(1.0, ls='--', color='black')
axs[1,1].set_ylim(0,)
axs[1,1].set_xlim(-6.5, 0.5)
axs[1,1].set_ylabel('Cumulative probability')

axs[2,1].plot(x, norm_pdf, '-', color='black', label='Original')
axs[2,1].plot(x, conv_const, '--', color='red', label='Pb loss')
axs[2,1].set_xlim(x_1, x_2)
axs[2,1].set_ylim(0,)
axs[2,1].legend()

axs[3,1].plot(x, np.cumsum(norm_pdf), '-', color='black', label='Original')
axs[3,1].plot(x, np.cumsum(conv_const), '--', color='red', label='Pb loss')
axs[3,1].set_xlim(x_1, x_2)
axs[3,1].set_ylim(0,1)
axs[3,1].set_xlabel('Pb*/U offset (%)')
axs[3,1].legend()

axs[0,2].plot(xk_mix, pb_loss_func_mix.pmf(xk_mix), 'ko', ms=12, mec='k')
axs[0,2].vlines(xk_mix, 0, pb_loss_func_mix.pmf(xk_mix), colors='k', lw=4)
axs[0,2].axhline(1.0, ls='--', color='black')
axs[0,2].set_ylim(0,)
axs[0,2].set_xlim(-6.5, 0.5)
axs[0,2].set_title('Isolated Pb loss')

axs[1,2].plot(x, cumulative_probs_mix, '-', color='k')
axs[1,2].axhline(1.0, ls='--', color='black')
axs[1,2].set_ylim(0,)
axs[1,2].set_xlim(-6.5, 0.5)
axs[1,2].set_ylabel('Cumulative probability')

axs[2,2].plot(x, norm_pdf, '-', color='black', label='Original')
axs[2,2].plot(x, conv_mix, '--', color='red', label='Pb loss')
axs[2,2].set_xlim(x_1, x_2)
axs[2,2].set_ylim(0,)
axs[2,2].legend()

axs[3,2].plot(x, np.cumsum(norm_pdf), '-', color='black', label='Original')
axs[3,2].plot(x, np.cumsum(conv_mix), '--', color='red', label='Pb loss')
axs[3,2].set_xlim(x_1, x_2)
axs[3,2].set_ylim(0,1)
axs[3,2].set_xlabel('Pb*/U offset (%)')
axs[3,2].legend()

for i in range(len(logitnorm_mu)):
    pdf_logitnorm = convFunc.Pb_loss_fun([logitnorm_mu[i], logitnorm_omega[i]], dist_type='logitnorm', x=x)
    conv_logitnorm = convolve(pdf_logitnorm, norm_pdf, mode='same')
    
    if i == 0:
        axs[0,3].set_title('Logit-normal Pb loss')
    
    axs[0,3].plot(x, pdf_logitnorm, '-', color=colors[i])
    axs[0,3].set_xlim(-6, 0)
    #axs[0,3].set_ylim(0,)
    axs[1,3].plot(x, np.cumsum(pdf_logitnorm), '-', color=colors[i])
    axs[1,3].set_xlim(-6, 0)
    axs[1,3].set_ylim(0,1)
    axs[2,3].plot(x, norm_pdf, '-', color='black')
    axs[2,3].plot(x, conv_logitnorm, '--', color=colors[i])
    axs[2,3].set_xlim(-15,10)
    axs[2,3].set_ylim(0,)
    axs[3,3].plot(x, np.cumsum(norm_pdf), '-', color='black')
    axs[3,3].plot(x, np.cumsum(conv_logitnorm), '--', color=colors[i])
    axs[3,3].set_ylim(0,1)
    axs[3,3].set_xlim(-15,10)
    
axs[3,3].set_xlabel('Pb*/U offset (%)')

plt.tight_layout()